<a href="https://colab.research.google.com/github/acse-dl1122/git_action_practice/blob/master/Copy_of_DL_module_Coursework_I_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?id=1dFgNX9iQUfmBOdmUN2-H8rPxL3SLXmxn" width="400"/>


---


### ***Name***: Dingo Luo
### ***CID***: 01765358

# Instructions:

Follow the instructions below to complete the coursework and submit it:

<br>

1. Complete your coursework using this provided Jupyter Notebook template (use Google Colab or your local machine if it has a GPU and/or sufficient computational power). Your copy of the notebook should be named: `yourusername_coursework_I.ipynb`. And don't forget to fill in the two fields at the top of this notebook with your name and CID.

<br>

2. Once you have completed your answers, upload your final notebook to the repo you got from the github classroom link. Make sure to have all the answers in there:

   - **All the cells in your final Jupyter Notebook should be executed before saving and uploading to github in order to have the output of the cells available in the uploaded version** (images you plot, training graphs generated with `livelossplot`, etc). We will not rerun code blocks in the notebooks, it is your responsibility to run them before uploading the notebook.

   - Add comments in the code to explain what you are doing at every step. 

   - All answers requiring written answers (ie, not code) should be in markdown blocks in the Jupyter Notebook. This provided Jupyter Notebook template has allocated blocks for the questions, but **you can add any coding or markdown blocks you need**.

<br>

3. The coursework is released on **Friday 9 December at 14:00h UK time**, and the answers have to be submitted on **Monday 12 December, 20:00h UK time**. We will not accept late submissions.

<br>

<br>

### The coursework consists of 5 questions you have to complete. You will find them below. 

- Questions 2 and 3 are very similar and you have to complete both of them, **you DO NOT have to choose one of them, you have to do them both.**

- You can use code seen in class, but indicate clearly when you do, and **make it clear (using comments or markdown blocks) what are your modifications**. This will influence your final marks.

- Your final marks will depend both on the quality of your results **AND** the justification, explanation and rationale of your implementations.

- Reference any publications and other materials that you use, but keep in mind that **your implementations should be original and not copied from any online resources (we will check for plagiarism)**. It is ok to use other resources to understand concepts and draw inspiration, but always reference them properly in the notebook.

- Finally, **justify your answers well, but focus on what the question is asking**. I will penalise lengthy answers that do not have any relation with the question being addressed.

<br>

---

<br>

## Imports

In [ ]:
!pip install livelossplot

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torch.autograd as autograd

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from livelossplot import PlotLosses
%matplotlib inline

import random

print('done')

<br>

---

<br>

## **Q1** - Download and prepare the Fashion MNIST dataset [10 points]

Get the `FashionMNIST` dataset from `torchvision.datasets`

- Inspect the dataset and format the data as you see fit to use it in your next questions. Explain the process you follow.

- Plot a matrix of images with 10 examples of each class.

- Is the dataset well balanced (does it have similar number of samples for each class)? Generate and present data to support your answer in any form you see fit.

### 1.1 Inspect the dataset and format the data as you see fit to use it in your next questions. Explain the process you follow.

In [ ]:
# Download the trainning data
train_data = FashionMNIST(
    root = 'data',
    train = True, 
    download = True,
    transform = ToTensor()
)

# Download the testing data
test_data = FashionMNIST(
    root = 'data',
    train = False, 
    download = True,
    transform = ToTensor()
)

train_data.data.shape, test_data.data.shape

In [ ]:
# Creating dataloade for both dataset
batch_size = 64

train_loader = DataLoader(dataset = train_data, batch_size = batch_size)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size)

In [ ]:
# The data have been splited into target (X) and label (y), which is to be used for the conditional VAE and conditional GAN. 

X_train = train_data.train_data.view(-1, 1, 28, 28).float()
Y_train = train_data.train_labels
X_test = test_data.test_data.view(-1, 1, 28, 28).float()
Y_test = test_data.test_labels;

### 1.2 Plot a matrix of images with 10 examples of each class.

In [ ]:
# Selecting 10 images from each class, and store their index into an dictionary
sample = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
}

size = 10

check = 0
for batch, (x, y) in enumerate(train_data):
    if len(sample[y]) < size:
        sample[y].append(batch)
        check += 1
    if check == size * 10:
        break

In [ ]:
all_sample_idx = []

for y in sample:
    for idx in sample[y]:
        all_sample_idx.append(idx)

In [ ]:
class_to_idx = list(train_data.class_to_idx)

plt.figure(figsize=(15, 15))

for i in range(10 * 10):
    plt.subplot(10, 10, i + 1)
    plt.axis("off")
    plt.imshow(X_train[all_sample_idx[i]].reshape(28, 28))
    plt.title('(%s) %s'%(
        int(Y_train[all_sample_idx[i]]),
        class_to_idx[Y_train[all_sample_idx[i]]]
    ))

plt.tight_layout(pad=1.1)
plt.show()

### 1.3 Is the dataset well balanced (does it have similar number of samples for each class)? Generate and present data to support your answer in any form you see fit.

In [ ]:
# checking the data
print(np.unique(train_data.targets, return_counts = True))
print(np.unique(test_data.targets, return_counts = True))

As we can see from the output, for training set and test set both include 10 classes, each class contains 6000 data for training and 1000 for testing. Threfore the data is **well balanced**. 

<br>

---

<br>

## **Q2** - Implement a **conditional VAE** [25 points]

Implement a VAE similar to the one we saw in class but with the following modifications:

- Your new network should generate samples of any class defined by the user, that is, when you run your decoder, you have to tell it which class you want it to generate. <br> *During the lectures we created a VAE that would generate samples of any class, we did not have control over which class would be generated. The exercise is to modify the network, training loops, and anything else you consider necessary, so that, once is trained, you can generate images of any particular class specified when you run your generative model.*

- Expand your network to include **at least two** of the following modifications. Choose them based on what you think will result in better network performance (make sure to justify your choices well, argumenting why did you decide to include your particular modifications):
  - Convolutional layers
  - Data augmentation 
  - Different activations
  - Different random seed
  - Other network modifications that are well justified. <br> *You have freedom to choose what to do here, but your marks will depend not only on the correct implementation but also on the justification of your choices.*

- Use your trained conditional VAE to **generate and plot 10 new samples of each class** by adding the class label to your random input (in any form you have implemented it).

- You **DO NOT** have to optimise any hyperparameters, but if you had to choose three hyperparameters to optimise:
  - Which ones would you choose and why?

Generate information of your training process (liveloss plots, or any other data that will help you understand your results and analyse them in the last question **Q4**).



### 2.1 Your new network should generate samples of any class defined by the user, that is, when you run your decoder, you have to tell it which class you want it to generate. 

In [ ]:
class cVAE_Encoder(nn.Module):  # The Encoder inherits the properties of nn.Module https://pytorch.org/docs/stable/generated/torch.nn.Module.html
    def __init__(self):
        '''
        Class contains the Encoder (image -> latent).
        '''
        super(cVAE_Encoder, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(794, 512),
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU()
        )

        self.layerMu = nn.Sequential(
            nn.Linear(128, 16)
        )

        self.layerSigma = nn.Sequential(
            nn.Linear(128, 16)
        )

    def forward(self, x, label):  # Custom pytorch modules should follow this structure 
        '''
        x: [float] the MNIST image
        label: [Tensor] label of the image after one-hot encode
        '''
        x = torch.flatten(x, start_dim = 1)  # Reshape the input into a vector (nD to 1D) 

        # append the one-hot label at the end of the vector. dim 784 -> 794
        x = torch.cat([x.cuda(), label.cuda()], 1)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        mu =  self.layerMu(x)
        sigma = torch.exp(self.layerSigma(x))
        return mu, sigma

In [ ]:
class cVAE_Decoder(nn.Module):  # The Encoder inherits the properties of nn.Module https://pytorch.org/docs/stable/generated/torch.nn.Module.html
    def __init__(self):
        '''
        Class contains the Decoder (latent -> image).
        '''

        super(cVAE_Decoder, self).__init__()

        self.layerLatent = nn.Sequential(
            nn.Linear(16, 128)
        )

        self.layer1 = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(512, 784),
            nn.Sigmoid() 
        )

    def forward(self, z, label):  # Custom pytorch modules should follow this structure 
        torch.cat([z.cuda(), label.cuda()], 1)

        z = self.layerLatent(z)
        z = self.layer1(z)
        z = self.layer2(z)
        z = self.layer3(z)

        # z = z[...,:-10]
        return z.reshape((-1,1,28,28))  # Reshape the vector into an image

print('done')


In [ ]:
class cVAE(nn.Module):
    def __init__(self, device):
        '''
        Class combines the Encoder and the Decoder with a VAE latent space.
        '''
        super(cVAE, self).__init__()
        self.device = device
        self.encoder = cVAE_Encoder()
        self.decoder = cVAE_Decoder()
        self.distribution = torch.distributions.Normal(0, 1)  # Sample from N(0,1)

    def sample_latent_space(self, mu, sigma):
        z = mu + sigma * self.distribution.sample(mu.shape).to(self.device)  # (1) Sample the latent distribution
        kl_div = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()  # A term, which is required for regularisation
        return z, kl_div

    def forward(self, x, label):
        '''
        x - [float] A batch of images from the data-loader
        '''

        mu, sigma = self.encoder(x, label)  ## (1) Generate the latent vectors Mu and Sigma
        z, kl_div = self.sample_latent_space(mu, sigma)  ## (2) Generate the latent vector sample 
        z = self.decoder(z, label)  ## (3) Generate the reconstructed image
        return z, kl_div

print('done')

In [ ]:
def evaluate(test_loss,autoencoder, dataloader):
    images, labels = next(iter(dataloader))
    images = images.view(images.size(0), 28*28)
    labels = F.one_hot(labels, num_classes = 10)
    loss = nn.MSELoss()
    loss_sum = []

    for idx, image in enumerate(images):
        recon, _ = autoencoder(image.unsqueeze(0).cuda(), labels[idx].unsqueeze(0).cuda()) 
        origin = torch.reshape(images[idx], (28, 28))
        pred = recon.cpu().detach().squeeze()
        loss_sum.append(loss(origin, pred))
    test_loss.append((sum(loss_sum)/len(loss_sum)).item())

print('done')

In [ ]:
def train(autoencoder, train_data, test_data, kl_div_on=True, epochs=10, device='cuda'):
    test_loss = []
    train_loss = []
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):  # Run data over numerous epochs
        epoch_loss = 0
        for batch, label in tqdm(train_data):  # Iterate over the batches of images and labels
            batch = batch.to(device)  # Send batch of images to the GPU
            opt.zero_grad()  # Set optimiser grad to 0
            label = F.one_hot(label, num_classes = 10)
            x_hat, KL = autoencoder(batch, label)  # Generate predicted images (x_hat) by running batch of images through autoencoder
            loss = ((batch - x_hat)**2).sum() + KL  # Calculate combined loss
            epoch_loss += loss.cpu().clone().detach().numpy()
            loss.backward()  # Back-propagate
            opt.step()  # Step the optimiser
        evaluate(test_loss, autoencoder, test_data)
        train_loss.append(epoch_loss)


        # plt.figure(figsize=(15, 9))
        # plt.plot(train_loss, label="train_loss")
        # plt.legend(loc='best')
        # plt.xlabel("epochs")
        # plt.ylabel("loss")
        # plt.title("train_loss vs. epochs")
        # plt.show()

    return autoencoder, test_loss, train_loss  # Return the trained autoencoder (for later analysis)

device = 'cuda'

dims_latent = 2  # Maybe increase this an try the t-sne algorithm for visualisation?!
model_cVAE = cVAE(device).to(device)
history_cVAE, test_loss_cVAE, train_loss_cVAE = train(model_cVAE, train_loader, test_loader, epochs=20, device=device)

print('done')

In [ ]:

plt.figure(figsize=(15, 9))
plt.plot(test_loss_cVAE, label="test_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE test loss")
plt.show()


In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(train_loss_cVAE, label="train_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE train loss")
plt.show()

In [ ]:
images, labels = next(iter(test_loader))
_, ax = plt.subplots(2, 5, figsize=[18.5, 6])

images = images.view(images.size(0), 28*28)
labels = F.one_hot(labels, num_classes = 10)


for n, idx in enumerate(torch.randint(0,images.shape[0], (5,))):
    recon, _ = history_cVAE(images[idx].unsqueeze(0).cuda(), labels[idx].unsqueeze(0).cuda()) 
    ax[0, n].imshow(torch.reshape(images[idx], (28, 28)).squeeze())
    ax[1, n].imshow(recon.cpu().detach().squeeze())

### 2.2 Expand your network to include at least two of the following modifications.
  - Convolutional layers
  - Data augmentation 
  - Different activations
  - Different random seed
  - Other network modifications that are well justified.

#### Different activations

The first implimentation that I would like to do is to change different activation functions. There are three common activation function is to be used in cVAE model:
- ReLU
- Sigmoid
- Tanh <br>

For the cVAE model above I have used ReLU activation function for each layer except the output layer, this is because the docoder network typically uses a non-linear activation function. When a model with all linear activation function, the output is also linear pattern so that does not have enough complexity.

In the code below I will change the ReLU activation function into Sigmoid and Tanh, and compare the result difference between three different activation function. 

In [ ]:
class cVAE_Encoder_Sigmoid(nn.Module):  # The Encoder inherits the properties of nn.Module https://pytorch.org/docs/stable/generated/torch.nn.Module.html
    def __init__(self):
        '''
        Class contains the Encoder (image -> latent).
        '''
        super(cVAE_Encoder_Sigmoid, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(794, 512),
            nn.Sigmoid()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Sigmoid()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.Sigmoid()
        )

        self.layerMu = nn.Sequential(
            nn.Linear(128, 16)
        )

        self.layerSigma = nn.Sequential(
            nn.Linear(128, 16)
        )

    def forward(self, x, label):  # Custom pytorch modules should follow this structure 
        '''
        x: [float] the MNIST image
        label: [Tensor] label of the image after one-hot encode
        '''
        x = torch.flatten(x, start_dim = 1)  # Reshape the input into a vector (nD to 1D) 

        # append the one-hot label at the end of the vector. dim 784 -> 794
        x = torch.cat([x.cuda(), label.cuda()], 1)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        mu =  self.layerMu(x)
        sigma = torch.exp(self.layerSigma(x))
        return mu, sigma

In [ ]:
class cVAE_Decoder_Sigmoid(nn.Module):  # The Encoder inherits the properties of nn.Module https://pytorch.org/docs/stable/generated/torch.nn.Module.html
    def __init__(self):
        '''
        Class contains the Decoder (latent -> image).
        '''

        super(cVAE_Decoder_Sigmoid, self).__init__()

        self.layerLatent = nn.Sequential(
            nn.Linear(16, 128)
        )

        self.layer1 = nn.Sequential(
            nn.Linear(128, 256),
            nn.Sigmoid()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.Sigmoid()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(512, 784),
            nn.Sigmoid() 
        )

    def forward(self, z, label):  # Custom pytorch modules should follow this structure 
        torch.cat([z.cuda(), label.cuda()], 1)

        z = self.layerLatent(z)
        z = self.layer1(z)
        z = self.layer2(z)
        z = self.layer3(z)

        #z = z[...,:-10]

        return z.reshape((-1,1,28,28))  # Reshape the vector into an image

print('done')


In [ ]:
class cVAE_Sigmoid(nn.Module):
    def __init__(self, device):
        '''
        Class combines the Encoder and the Decoder with a VAE latent space.
        '''
        super(cVAE_Sigmoid, self).__init__()
        self.device = device
        self.encoder = cVAE_Encoder_Sigmoid()
        self.decoder = cVAE_Decoder_Sigmoid()
        self.distribution = torch.distributions.Normal(0, 1)  # Sample from N(0,1)

    def sample_latent_space(self, mu, sigma):
        z = mu + sigma * self.distribution.sample(mu.shape).to(self.device)  # (1) Sample the latent distribution
        kl_div = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()  # A term, which is required for regularisation
        return z, kl_div

    def forward(self, x, label):
        '''
        x - [float] A batch of images from the data-loader
        '''

        mu, sigma = self.encoder(x, label)  ## (1) Generate the latent vectors Mu and Sigma
        z, kl_div = self.sample_latent_space(mu, sigma)  ## (2) Generate the latent vector sample 
        z = self.decoder(z, label)  ## (3) Generate the reconstructed image
        return z, kl_div

print('done')

In [ ]:
def train(autoencoder, train_data, test_data, kl_div_on=True, epochs=10, device='cuda'):
    test_loss = []
    train_loss = []
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):  # Run data over numerous epochs
        epoch_loss = 0
        for batch, label in tqdm(train_data):  # Iterate over the batches of images and labels
            batch = batch.to(device)  # Send batch of images to the GPU
            opt.zero_grad()  # Set optimiser grad to 0
            label = F.one_hot(label, num_classes = 10)
            x_hat, KL = autoencoder(batch, label)  # Generate predicted images (x_hat) by running batch of images through autoencoder
            loss = ((batch - x_hat)**2).sum() + KL  # Calculate combined loss
            epoch_loss += loss.cpu().clone().detach().numpy()
            loss.backward()  # Back-propagate
            opt.step()  # Step the optimiser
        evaluate(test_loss, autoencoder, test_data)
        train_loss.append(epoch_loss)

    return autoencoder, test_loss, train_loss  # Return the trained autoencoder (for later analysis)

device = 'cuda'

dims_latent = 2  # Maybe increase this an try the t-sne algorithm for visualisation?!
model_cVAE_Sigmoid = cVAE_Sigmoid(device).to(device)
history_cVAE_Sigmoid, test_loss_cVAE_Sigmoid, train_loss_cVAE_Sigmoid = train(model_cVAE_Sigmoid, train_loader, test_loader, epochs=20, device=device)

print('done')

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(test_loss_cVAE, label="test_loss ReLU")
plt.plot(test_loss_cVAE_Sigmoid, label="test_loss Sigmoid")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE test loss")
plt.show()

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(train_loss_cVAE, label="train_loss ReLU")
plt.plot(train_loss_cVAE_Sigmoid, label="train_loss Sigmoid")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE train loss")
plt.show()

In [ ]:
class cVAE_Encoder_Tanh(nn.Module):  # The Encoder inherits the properties of nn.Module https://pytorch.org/docs/stable/generated/torch.nn.Module.html
    def __init__(self):
        '''
        Class contains the Encoder (image -> latent).
        '''
        super(cVAE_Encoder_Tanh, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(794, 512),
            nn.Tanh()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Tanh()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.Tanh()
        )

        self.layerMu = nn.Sequential(
            nn.Linear(128, 16)
        )

        self.layerSigma = nn.Sequential(
            nn.Linear(128, 16)
        )

    def forward(self, x, label):  # Custom pytorch modules should follow this structure 
        '''
        x: [float] the MNIST image
        label: [Tensor] label of the image after one-hot encode
        '''
        x = torch.flatten(x, start_dim = 1)  # Reshape the input into a vector (nD to 1D) 

        # append the one-hot label at the end of the vector. dim 784 -> 794
        x = torch.cat([x.cuda(), label.cuda()], 1)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        mu =  self.layerMu(x)
        sigma = torch.exp(self.layerSigma(x))
        return mu, sigma

In [ ]:
class cVAE_Decoder_Tanh(nn.Module):  # The Encoder inherits the properties of nn.Module https://pytorch.org/docs/stable/generated/torch.nn.Module.html
    def __init__(self):
        '''
        Class contains the Decoder (latent -> image).
        '''

        super(cVAE_Decoder_Tanh, self).__init__()

        self.layerLatent = nn.Sequential(
            nn.Linear(16, 128)
        )

        self.layer1 = nn.Sequential(
            nn.Linear(128, 256),
            nn.Tanh()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.Tanh()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(512, 784),
            nn.Tanh() 
        )

    def forward(self, z, label):  # Custom pytorch modules should follow this structure 
        torch.cat([z.cuda(), label.cuda()], 1)

        z = self.layerLatent(z)
        z = self.layer1(z)
        z = self.layer2(z)
        z = self.layer3(z)

        # z = z[...,:-10]

        return z.reshape((-1,1,28,28))  # Reshape the vector into an image

print('done')


In [ ]:
class cVAE_Tanh(nn.Module):
    def __init__(self, device):
        '''
        Class combines the Encoder and the Decoder with a VAE latent space.
        '''
        super(cVAE_Tanh, self).__init__()
        self.device = device
        self.encoder = cVAE_Encoder_Tanh()
        self.decoder = cVAE_Decoder_Tanh()
        self.distribution = torch.distributions.Normal(0, 1)  # Sample from N(0,1)

    def sample_latent_space(self, mu, sigma):
        z = mu + sigma * self.distribution.sample(mu.shape).to(self.device)  # (1) Sample the latent distribution
        kl_div = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()  # A term, which is required for regularisation
        return z, kl_div

    def forward(self, x, label):
        '''
        x - [float] A batch of images from the data-loader
        '''

        mu, sigma = self.encoder(x, label)  ## (1) Generate the latent vectors Mu and Sigma
        z, kl_div = self.sample_latent_space(mu, sigma)  ## (2) Generate the latent vector sample 
        z = self.decoder(z, label)  ## (3) Generate the reconstructed image
        return z, kl_div

print('done')

In [ ]:
# Train the model
model_cVAE_Tanh = cVAE_Tanh(device).to(device)
history_cVAE_Tanh, test_loss_cVAE_Tanh, train_loss_cVAE_Tanh = train(model_cVAE_Tanh, train_loader, test_loader, epochs=20, device=device)

print('done')

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(test_loss_cVAE, label="test_loss ReLU")
plt.plot(test_loss_cVAE_Sigmoid, label="test_loss Sigmoid")
plt.plot(test_loss_cVAE_Tanh, label="test_loss Tanh")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE test loss")
plt.show()

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(train_loss_cVAE, label="train_loss ReLU")
plt.plot(train_loss_cVAE_Sigmoid, label="train_loss Sigmoid")
plt.plot(train_loss_cVAE_Tanh, label="train_loss Tanh")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE train loss")
plt.show()

As we can see, the ReLU activation function have better performance in terms of training speed and accuracy. Therefore we can keep the ReLU activation function as the best choice. 

#### Different random seeds

In cVAE, the random seed is initialized and to determines the sequence of random numbers that are generated by the model. This can have a significant impace on the performance and accuracy of the model.

Implimenting different random seeds in a CVAE allows the model to be trained and evaluated using multiple different sets of random numbers, which is going to provide a more robust estimate of the model's performance. 

Implimenting differnt random seeds is particularly useful when tring to improve the model's avility to generate new data, as it can help to ensure that the results are not overly influenced by the specific random seed that was used during training.

In summary, the use of different random seeds in a cVAE can provide several benefits, including improved generalization, reproducibility, and robustness.

In [ ]:
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True


In [ ]:
seed_train_loss = []
seed_test_loss = []
seed_number = []
for i in range(5):
    random_seed_model_name = f'history_cVAE_seed_{i}'

    set_seed(i)

    model_cVAE_seed = cVAE_Tanh(device).to(device)
    random_seed_model_name, test_loss_cVAE, train_loss_cVAE = train(model_cVAE, train_loader, test_loader, epochs=20, device=device)

    seed_number.append(i)
    seed_train_loss.append(train_loss_cVAE)
    seed_test_loss.append(test_loss_cVAE)
    print(i)
    print('=============================seed done======================')


In [ ]:
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

allDone()

In [ ]:
plt.figure(figsize=(15, 9))
for i in range(len(seed_number)):
    plt.plot(seed_test_loss[i], label=i)
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE test loss with different random seed")
plt.show()

In [ ]:
plt.figure(figsize=(15, 9))
for i in range(len(seed_number)):
    plt.plot(seed_train_loss[i], label=i)
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("cVAE train loss with different random seed")
plt.show()

As we can see from above, different random seed number do result in better loss for the training dataset, but for the testing data set, there have no much impact into the loss. Therefore

### 2.3 Use your trained conditional VAE to **generate and plot 10 new samples of each class** by adding the class label to your random input (in any form you have implemented it).


In [ ]:
with torch.set_grad_enabled(False):
    generation_class = torch.arange(10)
    generation_class = generation_class.repeat_interleave(10)
    gen_images = torch.FloatTensor(np.random.randn(100*28*28).reshape(-1,28,28)).cuda()
    gen_labels = F.one_hot(generation_class, num_classes = 10).cuda()
    generations, loss = history_cVAE(gen_images, gen_labels)

In [ ]:
def plot_gallary(images, labels, rows, cols, title):
    class_to_idx = list(train_data.class_to_idx)
    plt.figure(figsize=(15, 15))
    for i in range(rows * cols):
        plt.subplot(rows, cols, i + 1)
        plt.axis("off")
        plt.imshow(torch.reshape(images[i], (28, 28)).squeeze().cpu().clone().numpy())
        plt.title('(%s) %s'%(
            int(labels[i]),
            class_to_idx[labels[i]]
        ))
    plt.tight_layout(pad=1.1)
    plt.title(title)
    plt.show()
    
plot_gallary(generations, generation_class, 10, 10, 'Generated images from cVAE using random input and a label')

### 2.4 You **DO NOT** have to optimise any hyperparameters, but if you had to choose three hyperparameters to optimise:


The first hyperparameter that I would like to optimise is the size & the number of layers in the encoder and decoder. The size and number of layers is used to determine the capacity of the model to learn complex distributions. The correct size and number of layers can improve the model's ability to accurately encode and decode data. A smaller and more efficient cVAE is less computationally complex and can be trained more quickly, which made it more practical for use in real-world applications. 

The second hyperparameter that I want to optimise is the optimization algorithm. By selecting the approporate optimization algorithm, it is possible to train the model more efficiently and effectively, leading to better performance on the dataset. 

The third hyperparameter that I need to consider is the batch sizes. The batch size determines the number of samples that are processed by the model before the weights are updated. With a larger batch size result in accelerate trainning, but less stable gradient and higher generalization error. With smaller batch size can have better generalization but slower training. Overall, finding the optimal values for the batch size can help to balnace trade-offs and improve the performance of the cVAE

<br>

---

<br>

## **Q3** - Implement a **conditional GAN** that generates a user-defined class [25 points]

Implement a GANs similar to the one we saw in class but with the following modifications:

- Your new network should generate samples of any class defined by the user, that is, when you run your generator, you have to tell it which class you want it to generate. <br> *During the lectures we created a GAN that would generate samples of any class, we did not have control over which class would be generated. The exercise is to modify the network, training loops, and anything else you consider necessary, so that, once is trained, you can generate images of any particular class specified when you run your generative model.*

- Expand your network to include **at least two** of the following modifications. Choose them based on what you think will result in better network performance (make sure to justify your choices well, argumenting why did you decide to include your particular modifications):
  - Convolutional layers
  - Data augmentation
  - Different activations
  - Different random seed
  - Other network modifications that are well justified. <br> *You have freedom to choose what to do here, but your marks will depend not only on the correct implementation but also on the justification of your choices.*

- Use your trained conditional GAN to **generate and plot 10 new samples of each class** by adding the class label to your random input (in any form you have implemented it).

- You **DO NOT** have to optimise any hyperparameters, but if you had to choose three hyperparameters to optimise:
  - Which ones would you choose and why?
  - Would they be different from your choices in **Q2**?


Generate information of your training process (liveloss plots, or any other data that will help you understand your results and analyse them in the last question **Q4**).

### 3.1 Your new network should generate samples of any class defined by the user, that is, when you run your generator, you have to tell it which class you want it to generate.

In [ ]:
# class Generator(nn.Module):
#     def __init__(self, g_input_dim=100+10, g_output_dim=28*28):
#         super().__init__()       
#         self.fc1 = nn.Linear(g_input_dim, 256)
#         self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
#         self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
#         self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
#     # forward method
#     def forward(self, x, label): 
#         x = torch.cat([x.cuda(), label.cuda()], 1)
#         x = F.leaky_relu(self.fc1(x), 0.2)
#         x = F.leaky_relu(self.fc2(x), 0.2)
#         x = F.leaky_relu(self.fc3(x), 0.2)
#         return torch.tanh(self.fc4(x))

In [ ]:
# for testing
class Generator(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(100 + 10, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(256, 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(512, 1024),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(1024, 784),
        nn.Tanh()
    )

  def forward(self, z, label):
    # Concatenate noise and label
    z_label = torch.cat((z.cuda(), label.cuda()), dim=1)
    z = self.model(z_label)
    return z

In [ ]:
# class Discriminator(nn.Module):
#     def __init__(self, d_input_dim=28*28+10):
#         super().__init__()
#         self.fc1 = nn.Linear(d_input_dim, 1024)
#         self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
#         self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
#         self.fc4 = nn.Linear(self.fc3.out_features, 1)
    
#     # forward method
#     def forward(self, x, label):

#         x = x.view((bs,-1))
#         x = torch.cat([x.cuda(), label.cuda()], 1)

#         x = F.leaky_relu(self.fc1(x), 0.2)
#         x = F.dropout(x, 0.3)
#         x = F.leaky_relu(self.fc2(x), 0.2)
#         x = F.dropout(x, 0.3)
#         x = F.leaky_relu(self.fc3(x), 0.2)
#         x = F.dropout(x, 0.3)
#         return self.fc4(x)
    

In [ ]:
# for testing
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(784 + 10, 1024),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )

  def forward(self, x, label):
    # Concatenate image and label
    img_label = torch.cat((x.view(x.size(0), 784).cuda(), label.cuda()), dim=1)
    x = self.model(img_label)
    return x

In [ ]:
G = Generator().to(device)
D = Discriminator().to(device)

In [ ]:
# Define the hyperparameters

criterion = nn.BCELoss() 
z_dim = 100
bs = 100
lambda_gp = 10
n_critic = 5

# optimizer
lr = 0.0001

In [ ]:
train_loader = DataLoader(dataset = train_data, batch_size = bs)
test_loader = DataLoader(dataset = test_data, batch_size = bs)

In [ ]:
G_optimizer = torch.optim.Adam(G.parameters(), lr = lr)
D_optimizer = torch.optim.Adam(D.parameters(), lr = lr)

In [ ]:
def gp_term(x_real, x_fake, label):
    m = bs
    epsilon = torch.rand(m, 1).to(device)
    x_interp = (epsilon * x_real + (1 - epsilon) * x_fake).to(device)
    x_interp = Variable(x_interp, requires_grad = True) # so we can compute the grad
    D_interp = D(x_interp, label)
    
    grad_D_interp = autograd.grad(outputs = D_interp,
                 inputs = x_interp,
                 grad_outputs=torch.ones((bs, 1)).to(device),
                 create_graph=True, retain_graph=True, only_inputs=True)[0]
                 
    D_gp = (grad_D_interp.norm(2, dim=1) - 1) ** 2

    return D_gp

In [ ]:
def D_train(x, lambda_gp, label):
    #-------------- Function of the discriminator training -------------------#
    D.train()
    D_optimizer.zero_grad()

    # D_real 
    x_real = x.view(-1, 28*28)
    x_real= Variable(x_real.to(device))
    D_real = D(x_real, label)

    # D_fake
    # sample vector and produce generator output
    z = Variable(torch.randn(bs, z_dim).to(device))
    x_fake = G(z, label)
    D_fake = D(x_fake, label)

    # D_gp
    D_gp = gp_term(x_real, x_fake, label)
    
    # combine the losses
    D_loss = D_fake.mean() - D_real.mean() + lambda_gp * D_gp.mean()

    # model update 
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [ ]:
def G_train(x, label):
    #-------------- Function of the generator training -------------------#
    G.train()
    G_optimizer.zero_grad()

    # sample vector and produce generator output
    z = Variable(torch.randn(bs, z_dim).to(device))
    x_fake = G(z, label)

    # D_output for x_fake
    D_output = D(x_fake, label)

    # G loss
    G_loss = - D_output.mean() 

    # model update 
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

In [ ]:
n_epoch = 100 # 200 epochs is about 40 minutes
groups = {'Loss': ['D_Loss', 'G_Loss']}
liveloss = PlotLosses(groups=groups)

for epoch in range(1, n_epoch+1):  
  D_losses, G_losses = [], []
  logs = {}
  
  for batch_idx, (x, y) in enumerate(train_loader):
    y = F.one_hot(y, num_classes = 10)
    logs['D_Loss'] = D_train(x, lambda_gp, y) 
    if (batch_idx % n_critic == 0):
      logs['G_Loss'] = G_train(x, y)
  liveloss.update(logs)
  liveloss.draw()

  # save every 20th epochs
  if(np.mod(epoch, 10) == 0):
    torch.save(G.state_dict(), "./Generator_{:03d}.pth".format(epoch))


In [ ]:
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

allDone()

### 3.2 Expand your network to include **at least two** of the following modifications
  - Convolutional layers
  - Data augmentation
  - Different activations
  - Different random seed
  - Other network modifications that are well justified.


#### Different random seed

The first modification that I would like to impliment is the random seed. This is because the random noise vector is added to the generator, and the generator use it as a starting point to generate new data samples. When the random noise is too close to a real image(data), the discriminator will have a difficult time determining whether the output is real or fake. Therefore changing the random seed are very important for the cGAN model, 

In [ ]:
def train_cGAN(n_epoch, seed):
    groups = {'Loss': ['D_Loss', 'G_Loss']}
    liveloss = PlotLosses(groups=groups)

    for epoch in range(1, n_epoch+1):  
        D_losses, G_losses = [], []
        logs = {}
    
        for batch_idx, (x, y) in enumerate(train_loader):
            y = F.one_hot(y, num_classes = 10)
            logs['D_Loss'] = D_train(x, lambda_gp, y) 
            if (batch_idx % n_critic == 0):
                logs['G_Loss'] = G_train(x, y)
        liveloss.update(logs)
        liveloss.draw()

    
        # save every 20th epochs
        if(np.mod(epoch, 10) == 0):
            torch.save(G.state_dict(), "./Generator_{:03d}_{:03d}.pth".format(epoch, seed))


In [ ]:
for i in range(2):
    set_seed(i)
    train_cGAN(10, i)

In [ ]:
G.load_state_dict(torch.load("./Generator_{:03d}_{:03d}.pth".format(10, 1)))

#### Different activation function

From the code above, I have implimented leakyReLU for all the layers as the activation function. I would also like to impliment ReLU function, which introduce the non-linearity into the network and allowing complex model distributions. The Sigmoid function can also be ussed to at the output layer to squash the output values between 0 and 1, making it easier to imterpret the output as a probability. The Tanh function is similar to Sigmoid but squash the output between -1 and 1, which is less useful in our case. 


### 3.3 Use your trained conditional GAN to **generate and plot 10 new samples of each class** by adding the class label to your random input (in any form you have implemented it).


In [ ]:
# from torchvision.utils import save_image
set_seed(0)

epoch = 100
G.load_state_dict(torch.load("./Generator_{:03d}.pth".format(epoch)))


with torch.no_grad():
    generation_class = torch.arange(10)
    generation_class = generation_class.repeat_interleave(10)
    gen_labels = F.one_hot(generation_class, num_classes = 10).cuda()
    test_z = Variable(torch.randn(100, z_dim).to(device))
    generated = G(test_z, gen_labels)

plot_gallary(generated.view(generated.size(0), 28, 28).cpu(), generation_class, 10, 10, 'Epoch = {:03d}'.format(epoch))


improve the generalization of the model. 


### 3.4 You **DO NOT** have to optimise any hyperparameters, but if you had to choose three hyperparameters to optimise:
  - Which ones would you choose and why?
  - Would they be different from your choices in **Q2**?text block for your answers

The first hyperparameter that I want to optimise is the learning rate. As we can see from the loss_plot for the cGAN above, the G_loss are having huge changes for each epochs, and seems to stuck at one place. Optimizing the learning rate can control the speed of updating weight during the trainning and can help improve the performance of the model by ensuring that the parameters are updated in a way that allows the model to learn effeitively from the data. What's more, find the optimal learning rate can help preventr overfitting. <br>
This is different from Q2 is because for Q2, the loss of the model decrease smoothly as the epochs increase. 

The other two hyperparameters that I want to optimmise is the batch size and the structure of the layers. The reasons are the same for questions 2 with cVAE. 

<br>

---

<br>

## **Q4** - Analysis and comparison of results [20 points]

Analyse and compare the results you have obtained with the two networks above.

- Use the data you generated training the two networks above to discuss the results you have generated in **Q2** and **Q3**.
- Is there a network that performs better than the other? Why do you think that is?
- What could you do to improve your results on each of the networks you have implemented? ***\[limit your answer to this question to 150 words\]***

**IMPORTANT NOTE**: even if you did not complete **Q2** and **Q3**, make sure to include a discussion with any partial results or other considerations regarding the performance of your implemented methods.

In [ ]:
### your code here, if you need any (add as many code blocks as you need)

text block for your answers

<br>

---

<br>

## **Q5** - Modify your dataset to create data imbalance and retrain **one** of your conditional models [20 points]

Modify the `FashionMNIST` dataset to eliminate 90% of the images corresponding to one of the classes in the dataset (whichever you want).

Now, **choose one of the two questions below (a or b)**:

- **Q5-a) Retrain your conditional VAE from scratch** with this new dataset, and after training, use it to generate 10 images for the class you have decimated.

- **Q5-b) Retrain your conditional GAN from scratch** with this new dataset, and after training, use it to generate 10 images for the class you have decimated.

- Compare the results of this new images generated with the modified dataset with the original results from questions **Q2** or **Q3** (depending of whether you have chosen **Q5-a** or **Q5-b**). What do you observe? Describe your interpretation of this comparison.

In [ ]:
# your code here (add as many code blocks as you need)

text block for your answers

In [ ]:
### your code here, if you need any (add as many code blocks as you need)

text block for your answers

# Testing/

In [ ]:
LR_G=0.0001
LR_D=0.0001
Batch_size=100